In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Flatten,MaxPooling2D,Input,BatchNormalization,Dropout,Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard ,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np


In [ ]:

input = Input(shape=(48,48,1))
x = Conv2D(filters=256,kernel_size=3,activation='relu',padding='same')(input)

x = Conv2D(filters=512,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)

#
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)

x = Conv2D(filters=384,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)

x = Conv2D(filters=192,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)


x = Conv2D(filters=384,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)

x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)

x = Flatten()(x)

x = Dense(256,activation='relu')(x)
x = BatchNormalization()(x)

x = Dropout(0.3)(x)
x = Dense(7,activation='softmax')(x)

model=Model(input,x)

model.summary()



In [ ]:

train=r"/kaggle/input/fer2013/train"
val = '/kaggle/input/fer2013/test'
train_datagen = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range = 10,
    zoom_range = 0.1,
    horizontal_flip = True,
     width_shift_range=0.1,
    height_shift_range=0.1
   )

train_generator = train_datagen.flow_from_directory(
    directory=train,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=128,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
test_datagen = ImageDataGenerator(
    rescale=1 / 255.0,)

valid_generator = test_datagen.flow_from_directory(

    directory=val,
    target_size=(48, 48),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=128,
    shuffle=False,
    seed=42
)

In [ ]:
class_labels = list(train_generator.class_indices.keys())
class_counts = train_generator.classes

fig, ax = plt.subplots()
ax.hist(class_counts, bins=len(class_labels))
ax.set_xticks(range(len(class_labels)))
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.set_xlabel('Class')
ax.set_ylabel('Number of images')
plt.show()

In [ ]:
class_labels = list(valid_generator.class_indices.keys())
class_counts = valid_generator.classes

fig, ax = plt.subplots()
ax.hist(class_counts, bins=len(class_labels))
ax.set_xticks(range(len(class_labels)))
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.set_xlabel('Class')
ax.set_ylabel('Number of images')
plt.show()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath="/kaggle/working/new/CheckPoint.h5",
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )
early_stopping = EarlyStopping(monitor='val_loss', patience=20,verbose=1)
tensorboard = TensorBoard(log_dir="/kaggle/working/")

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
with tf.device('/GPU:0'):
    history = model.fit(train_generator,
                                  batch_size=128,
                                  validation_data=valid_generator,
                                  epochs=100,
                                  callbacks=[early_stopping, checkpoint,tensorboard],
                                  shuffle=False,
                                  verbose=1
                                 )

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)

ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)

ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)

ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)

ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)

plt.savefig('final_cnn_loss_accuracy.png', bbox_inches='tight');

In [ ]:
model.evaluate(valid_generator)

In [ ]:
predictions = model.predict(valid_generator, steps=valid_generator.samples // 128 + 1)
print(predictions[0])

In [ ]:
np.argmax(predictions[0])

In [ ]:
predicted_classes =[]
for pred in predictions:
    predicted_classes.append(np.argmax(pred))




In [ ]:
true_classes = valid_generator.classes
print(true_classes[5000])

In [ ]:
cm = confusion_matrix(true_classes, predicted_classes)


In [ ]:

cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
target_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(cm_norm, cmap=plt.cm.Blues, alpha=0.3)
for i in range(cm_norm.shape[0]):
    for j in range(cm_norm.shape[1]):
        ax.text(x=j, y=i, s=format(cm_norm[i, j], '.2f'),
                ha='center', va='center', color='black', fontsize=14)
plt.xticks(np.arange(len(target_names)), target_names, fontsize=14)
plt.yticks(np.arange(len(target_names)), target_names, fontsize=14)
plt.xlabel('Predicted label', fontsize=14)
plt.ylabel('True label', fontsize=14)
plt.title('Confusion matrix', fontsize=16)
plt.show()